# Word2Vec as a Model of Interactions between Words

## Word2Vec Introduction

The [word2vec](https://en.wikipedia.org/wiki/Word2vec) model was proposed to vectorize words. A word is a string. It cannot be "computed" on a computer. We have to encode words to vectors. One direct way of encoding is called one-hot. That is, given a vocabulary which is a list of words, the i-th word has vector $x$ with $x^i$ = 1 and all other components vanish, like $(0, \cdots, 0, 1, 0, \cdots, 0)$. One-hot encoding is not very efficient since its dimension equals to the vocabulary size. But the vocabulary may be quite large. This motives the idea of word2vec, that is, encoding words to dense vectors with a small dimension.

The basic idea behind word2vec is modeling the probability of appearance of two given words $w_1$ and $w_2$ as a neighbour in a corpus. Given two words $w_1$ and $w_2$ with vectors $x_1$ and $x_2$ respectively, the probability of being neighbour is assumed to be

$$ p_{\text{neighbour}} (w_1, w_2) \propto \exp(x_1 \cdot x_2). $$

So, for word2vec model, the learning task find a vector for each word so that the $p_{\text{neighbour}}$ fits the real data.

## Word2Vec as Interactions between Words

If words have been one-hot encoded, then for each one-hot encoded word $w_i$, its vector is given by $x_i = W \cdot w_i$. The matrix $W$ has dimension $(E, V)$ where $E$ represents the word-vector dimension and $V$ the vocabulary size. Then, it can be derived directly that

$$ x_1 \cdot x_2 = \frac{1}{2} u^t \cdot A \cdot u, $$

where

$$ u := w_1 + w_2 $$

and

$$ A := W^t \cdot W - \textrm{diag} (W^t \cdot W). $$

The matrix $A$ has dimension $(V, V)$. It is symmetric, with vanished diagonal elements. It is recognized as a Boltzmann machine with the energy given by $E(u; A) := -(1/2) u^t \cdot A \cdot u$ and unit temperature. Fitting a Boltzmann machine is minimizing the loss

$$ L(W) = E(w_1, w_2; W) - E(\tilde{w}_1, \tilde{w}_2; W), $$

for any two neighboured words (one-hot encoded) $(w_1, w_2)$ and two "fantasy" words $(\tilde{w}_1, \tilde{w}_2)$. In this expression, we have re-write $E$ in $w$s and $W$. The key point is ensuring that $E(w_1, w_2; W) > E(\tilde{w}_1, \tilde{w}_2; W)$ is more probable than the inverse. In this way, the $W$ is adjusted so that the $(w_1, w_2)$ is going to be a local minimum of the energy.

Boltzmann machine ensures this by sampling $u' \sim \text{Bernoulli}(\sigma(A \cdot u))$, where $\sigma$ is the sigmoid function. Generally, the sampled is a multi-hot vector, representing a collection of different word-indices, $(w_{i_1}, \ldots, w_{i_K})$. The energy of $u'$ then comes to be

$$ E(u'; A) = -\frac{1}{2} u^t \cdot A \cdot u = \sum_{i_m} \sum_{i_n \neq i_m}  E(\tilde{w}_{i_m}, \tilde{w}_{i_n}; W). $$

The summation runs over all the pairs of $(\tilde{w}_{i_m}, \tilde{w}_{i_n})$ where $i_m$ and $i_n$ are distinct. The pairs may become too many to compute. In addition, the $K$ varies for each datum. So, it is hard to compute in TensorFlow (we will employ) wherein tensor shapes are static. We shall select a fixed number of pairs from them, while ensuring that the fantasy energy is no less than the real energy. The greater the $(A \cdot u)_k$ is for some word $w_k$, the more probable it will be sampled. So, we are to select a pair in which words are more likely being sampled. To do this, we consider the categorical distribution

$$ p_{\alpha} = \frac{ \exp(\sum_{\beta} A_{\alpha \beta} u^{\beta} / T) }{ \sum_{\alpha'} \exp(\sum_{\beta'} A_{\alpha' \beta'} u^{\beta'} / T) }, $$

where $T$ is a positive number that characterizes the randomness. It is a categorical distribution with alphabet size $V$. It is indicated that we shall sample two word-indices based on this distribution as the selected pair. If the fantasy energy is no less than the real energy, then the loss will be non-negative.

In addition, the component $(A \cdot u)_k$ can be reduced to

$$ ( z_{k m} - \delta_{k m} z_{k m} ) + ( z_{k n} - \delta_{k n} z_{k n} ), $$

where $z_{k m}$ represents the inner product between word-vectors of $x_k$ and $x_m$.

## Global Configurations

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python import keras
from collections import Counter

Configurations are given as follow. The vector dimension employed by the original word2vec paper is `300`.

In [2]:
NEIGHBOURS = 2  # window size used for generating dataset.
VOCAB_SIZE = 2 ** 14  # vocabulary size.
VECTOR_DIM = 300  # dimension of word-vector.
BATCH_SIZE = 128  # batch size of training.

In addition, we add other global parameters as switchers for several experiments. We will test the case where $L(W) = E(w_1, w_2; W)$, that is, the fantasy energy is not involved. Also the case where fantasy data is sampled randomly and uniformly from the vocabulary. The results are summarized in the conclusion.

In [3]:
TEST_WITHOUT_FANTASY_ENERGY = False
TEST_UNIFORM_SAMPLE_FANTASY = False

## Text Data

The original word2vec model is trained on the "text8" dataset. It is preprocessed and can be found on [internet](https://mattmahoney.net/dc/text8.zip). It is a zip file. By unzipping, we get a text file named `text8`. This is the preprocessed data that we can use directly, except for excluding words with single character.

In [4]:
%%bash
if [ ! -f text8 ]; then
    wget  https://mattmahoney.net/dc/text8.zip
    unzip text8.zip
    rm text8.zip
fi

In [5]:
with open('text8', 'r') as f:
    text8 = [word for word in list(f)[0].split(' ') if len(word) > 1]

There are lots of different words in the "text8" text. We shall limit the vocabulary used for building our model. For this purpose, we employ the most frequent words.

In [6]:
%%time
counter = Counter(text8)
vocab = {}
for i, (word, _) in enumerate(counter.most_common(VOCAB_SIZE)):
    vocab[word] = i

CPU times: user 3.88 s, sys: 47.8 ms, total: 3.93 s
Wall time: 5.47 s


In [7]:
id_to_word = {i: w for w, i in vocab.items()}

Now, we construct the collection of pairs of center word (called "target" in the original paper of word2vec) and its neighbour (called "context") in the corpus.

In [8]:
%%time
targets = []
contexts = []
for (i, target) in enumerate(text8[NEIGHBOURS:-NEIGHBOURS]):
    for j in range(i-NEIGHBOURS, i+NEIGHBOURS+1):
        if j == i: continue
        context = text8[j]
        targets.append(target)
        contexts.append(context)

CPU times: user 27.9 s, sys: 671 ms, total: 28.6 s
Wall time: 30.4 s


While converting from word to its index in the vocabulary, we have to drop the pairs in which there is at least one word that is absent in the vocabulary.

In [9]:
%%time
target_ids = []
context_ids = []
for w1, w2 in zip(targets, contexts):
    if w1 not in vocab or w2 not in vocab:
        continue
    target_ids.append(vocab[w1])
    context_ids.append(vocab[w2])

# List -> np.ndarray -> Dataset is much faster than List -> Dataset.
target_ids = np.asarray(target_ids, dtype='int32')
context_ids = np.asarray(context_ids, dtype='int32')

CPU times: user 29.8 s, sys: 674 ms, total: 30.4 s
Wall time: 30.5 s


In [10]:
target_ids.shape, context_ids.shape

((57706802,), (57706802,))

Now, convert the processed data to TensorFlow's dataset protocol for training.

In [11]:
ds = tf.data.Dataset.from_tensor_slices((target_ids, context_ids))

Let see some instances.

In [12]:
for x, y in ds.batch(5).take(1):
    tf.print('x: ', x)
    tf.print('y: ', y)

x:  [10 10 10 10 180]
y:  [19 13 3054 10 13]


## Model Implementation

In [13]:
class Word2Vec:
    """Word2Vec as a model of interactions between words.

    Args:
        vocab_size: Integer for the vocabulary size.
        vector_dim: Integer for the word-vector dimension.
        T: Positive float for the randomness in generating fantasy data.
    """

    # We use B for batch size, V for vocabulary size, and D for vector dimension.

    def __init__(self, vocab_size, vector_dim, T=1e-3):
        self.vocab_size = vocab_size
        self.vector_dim = vector_dim
        self.T = T

        W_init = tf.random.uniform([vocab_size, vector_dim], dtype=tf.float32,
                                   minval=-0.05, maxval=0.05)
        self.W = tf.Variable(W_init)  # (V, D)

    def __call__(self, x):
        return tf.nn.embedding_lookup(self.W, x)

    def energy(self, pair):
        x, y = pair
        return -tf.reduce_sum(self(x) * self(y), axis=1)

    def sample_fantasy(self, pair):
        if TEST_UNIFORM_SAMPLE_FANTASY:
            batch_size = tf.shape(pair[0])[0]
            samples = tf.random.uniform(
                shape=[batch_size, 2], maxval=self.vocab_size, dtype=tf.int32)
            return tf.unstack(samples, axis=1)

        def get_logits(x):
            raw_logits = tf.matmul(self(x), tf.transpose(self.W))  # (B, V)
            indices = tf.stack([tf.range(tf.shape(x)[0]), x], axis=1)
            update = tf.zeros(tf.shape(x))
            return tf.tensor_scatter_nd_update(raw_logits, indices, update)
        x, y = pair
        logits = get_logits(x) + get_logits(y)  # (B, V)
        # Sample two samples by probability proportional to `exp(logits / self.T)`.
        samples = tf.random.categorical((logits / self.T), 2, dtype=tf.int32)
        return tf.unstack(samples, axis=1)

    def loss(self, real_pair, fantasy_pair):
        if TEST_WITHOUT_FANTASY_ENERGY:
            return tf.reduce_mean(self.energy(real_pair))

        return tf.reduce_mean(self.energy(real_pair) - self.energy(fantasy_pair))

    def get_train_step(self, optimizer):
        step = tf.Variable(0, dytpe=tf.int32)

        @tf.function
        def train_step(real_pair):
            fantasy_pair = self.sample_fantasy(real_pair)

            # Compute loss and its gradient, and optimize.
            with tf.GradientTape() as gt:
                loss_value = self.loss(real_pair, fantasy_pair)
            grads = gt.gradient(loss_value, self.W)
            # The gradient to the weights in embedding layer is treated as sparse,
            # Convert sparse to dense for optimizer.
            grads = tf.convert_to_tensor(grads)
            optimizer.apply_gradients([(grads, self.W)])

            step.assign_add(1)
            return loss_value

        return train_step, step

## Model Training

In [14]:
model = Word2Vec(VOCAB_SIZE, VECTOR_DIM)
optimizer = keras.optimizers.gradient_descent_v2.SGD()
train_step, step = model.get_train_step(optimizer)

In [15]:
process_bar = keras.utils.generic_utils.Progbar(len(ds.batch(BATCH_SIZE)))
for real_pair in ds.shuffle(10000).batch(BATCH_SIZE):
    loss_value = train_step(real_pair)
    process_bar.update(current=tf.cast(step, tf.float32),
                       values=[('loss', loss_value)])

450835/450835 [==============================] - 2239s 5ms/step - loss: 0.0586


In [16]:
model.W

<tf.Variable 'Variable:0' shape=(16384, 300) dtype=float32, numpy=
array([[-1.52643174e-02,  1.53087750e-02,  5.68150263e-03, ...,
        -1.95565750e-03,  8.43332428e-03,  3.50515801e-03],
       [-2.05261614e-02,  7.04018353e-03,  2.99878791e-03, ...,
        -2.86304182e-03, -5.76683437e-04,  6.45227777e-03],
       [-1.05065675e-02,  7.78524857e-03, -3.08963936e-05, ...,
        -6.58661174e-03,  2.18126252e-02,  8.18720181e-03],
       ...,
       [ 1.33575173e-02, -1.60282943e-02,  1.47384610e-02, ...,
        -1.80412363e-02, -1.86773334e-02,  1.88402813e-02],
       [-3.38749439e-02, -1.52752735e-02, -1.36089744e-02, ...,
         2.59806234e-02,  4.37434763e-02,  4.09261100e-02],
       [-3.05379201e-02, -2.03030510e-03,  6.38827449e-03, ...,
         6.64780149e-04, -4.22020033e-02, -1.93954725e-02]], dtype=float32)>

## Evaluation

To evalute the model, we consider the $k$ words that are closest to a given word. The relation between these words shall be meaningful.

In [17]:
def get_closest_k(model, word, k):
    vector = model([vocab[word]])  # (1, D)
    distances = tf.matmul(vector, tf.transpose(model.W))  # (1, V)
    _, top_ids = tf.math.top_k(distances, k=k)
    return top_ids.numpy()

In [18]:
for word in ('world', 'boy', 'zero', 'sun', 'music', 'the', 'to'):
    closest_indices = get_closest_k(model, word, 5)
    print(f'{word}: {", ".join([id_to_word[idx] for idx in closest_indices[0,:]])}\n')

world: world, war, international, between, us

boy: boy, filed, lighthouse, amplifier, kilogram

zero: zero, two, five, one, isbn

sun: sun, seemingly, janet, unpleasant, attention

music: music, classical, between, desired, garden

the: the, of, and, that, in

to: to, of, be, he, according



## Conclusion

From this simple evaluation, it has been found that the word2vec re-implemented from the aspect of interaction reveals some deeper relations of words.

If we drop the contribution in the loss from the fantasy data, the training fails in such a way that only the most frequent words (like "the", "of", and "in") appear as the closest for any word.

If we sample fantasy data by uniform sampler, the training also fails in the previous way.